In [2]:
# import libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
from sklearn.preprocessing import StandardScaler

print('Libraries imported successfully.')

Libraries imported successfully.


In [3]:
# get the coordinates of Helsinki with geopy
city = 'Helsinki'

geolocator = Nominatim(user_agent="helsinki_explorer")
location_helsinki = geolocator.geocode(city)
latitude_helsinki = location_helsinki.latitude
longitude_helsinki = location_helsinki.longitude
print('The geograpical coordinate of Helsinki are {}, {}.'.format(latitude_helsinki, longitude_helsinki))

The geograpical coordinate of Helsinki are 60.1674086, 24.9425683.


In [4]:
# set credentials for foursquare API
CLIENT_ID = '0ODKNL0XSP1A5NWJHPGJGIYRDISXDGXMACR23Z33IHZZZ0BA'
CLIENT_SECRET = 'VW2PTICIJXTOWDSPJG30TU0V0IO5CSK1IMUOI0ES1RACR0UC'
VERSION = '20180605'

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: 0ODKNL0XSP1A5NWJHPGJGIYRDISXDGXMACR23Z33IHZZZ0BA
CLIENT_SECRET:VW2PTICIJXTOWDSPJG30TU0V0IO5CSK1IMUOI0ES1RACR0UC


In [5]:
# get venues around the coordinates of Helsinki with foursquare explore API
LIMIT = 1000
radius = 5000
category_food = '4d4b7105d754a06374d81259' # get only venues of food category

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude_helsinki, 
    longitude_helsinki, 
    radius, 
    LIMIT,
    category_food)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c8370671ed2196e48ce6b68'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Helsinki',
  'headerFullLocation': 'Helsinki',
  'headerLocationGranularity': 'city',
  'query': 'food',
  'totalResults': 248,
  'suggestedBounds': {'ne': {'lat': 60.212408645000046,
    'lng': 25.03285766712385},
   'sw': {'lat': 60.12240855499996, 'lng': 24.852278932876153}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b865be2f964a520548731e3',
       'name': 'Café Trocadero',
       'location': {'address': 'Uudenmaankatu 2',
        'lat': 60.16510279120757,
        'lng': 24.94249786637479,
        'labeledLatLngs': [{'label': 'display',
          'lat': 60.165

In [6]:
# define function that gets the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [7]:
# get specific elements from the json results and put them into dataframe
venues = results['response']['groups'][0]['items']

helsinki_food_venues = json_normalize(venues)

# get specific columns
specific_columns = ['venue.name', 'venue.id', 'venue.categories', 'venue.location.lat', 'venue.location.lng', 'venue.location.postalCode']
helsinki_food_venues = helsinki_food_venues.loc[:, specific_columns]

# get categories
helsinki_food_venues['venue.categories'] = helsinki_food_venues.apply(get_category_type, axis=1)

# clean columns
helsinki_food_venues.columns = [col.split(".")[-1] for col in helsinki_food_venues.columns]

helsinki_food_venues.head()

,name,id,categories,lat,lng,postalCode
0,Café Trocadero,4b865be2f964a520548731e3,Mediterranean Restaurant,60.165103,24.942498,00120
1,Patisserie Teemu & Markus,4ddd2d44b0fba481fc927360,Bakery,60.167899,24.938190,00100
2,Penny,56bcc3c1498e7be180d65d6b,American Restaurant,60.164708,24.941308,00120
3,Gaijin,4dad421e93a070582bab90c0,Asian Restaurant,60.165949,24.941759,00120
4,Ravintola Vinkkeli,56e55e7d498e85d4fbfecd17,Scandinavian Restaurant,60.164107,24.946349,00130


In [8]:
helsinki_food_venues.shape

(100, 6)

In [9]:
# function for getting more detailed venue data from foursquare venue API

def getVenueData(ids):
    
    venues_data_list=[]
    
    for id in ids:
        url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            id,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)
        
        results = requests.get(url).json()['response']['venue']
                
        if 'price' in results:
            venues_data_list.append([(
                id, 
                results['stats']['tipCount'], 
                results['likes']['count'],
                results['rating'],
                results['price']['tier'])])         
            
            print(results['stats']['tipCount'])
            print(results['likes']['count'])
            print(results['rating'])
            print(results['price']['tier'])
            print("")
            
        else:
            venues_data_list.append([(
                id, 
                results['stats']['tipCount'], 
                results['likes']['count'],
                results['rating'])])
            
            print(results['stats']['tipCount'])
            print(results['likes']['count'])
            print(results['rating'])
            print("")         
            
    venues_data_df = pd.DataFrame([item for venues_data_list in venues_data_list for item in venues_data_list])
    venues_data_df.columns = ['Id', 
                'Tip Count', 
                'Likes',
                'Rating',
                'Price tier']
        
    return(venues_data_df)

In [10]:
helsinki_food_venues_data = getVenueData(ids=helsinki_food_venues['id'])

33
91
9.1
2

30
56
9.0
1

22
89
9.1
2

98
255
8.9
3

13
43
9.2
2

28
92
8.9
2

31
111
8.7
2

44
133
9.1
3

15
78
9.0
2

8
37
8.9
3

60
179
8.7
2

25
105
9.1
2

118
376
8.9
2

11
32
9.2
2

153
517
8.6
2

99
252
9.2
2

31
83
8.8
1

77
266
8.6
2

39
173
8.7
2

15
60
8.6
2

13
40
9.0
2

26
113
9.0
2

6
24
8.8
1

78
190
8.7
2

26
74
8.8
1

5
18
8.4
2

33
113
8.6
2

13
45
8.5
2

0
22
8.4
1

15
56
8.3
2

79
234
9.0
3

9
50
8.7
2

23
95
9.1
2

90
187
8.5
2

88
241
8.4
2

30
98
9.0
1

146
480
9.3
3

83
195
8.7
2

34
92
8.6
1

6
25
9.0
2

26
56
8.5
2

32
115
8.5
2

26
60
9.1
2

6
50
9.0
1

18
64
8.9
2

28
67
8.9
2

40
113
8.3
3

41
265
8.8

4
21
8.3
2

47
98
8.6
3

34
85
8.8
3

10
27
8.2

63
156
8.4
3

9
28
8.3
1

20
60
8.6
1

18
76
9.2

9
19
8.4
3

11
22
8.1
1

9
33
8.7
2

11
46
8.3

41
122
8.4
2

2
6
8.4
1

32
136
8.3
2

37
102
8.3
3

44
147
8.7
2

23
82
8.1
2

54
134
8.0
2

49
126
8.4
2

15
52
8.1
1

33
143
8.2
3

16
70
8.6
2

55
136
8.2
2

123
568
8.1
2

29
65
8.0
4

46
200
8.0
3

31
83
8.5


In [11]:
# add new columns from fourquare venue API to dataframe that included columns from foursquare explore API
helsinki_food_venues['Tip Count'] = helsinki_food_venues_data['Tip Count']
helsinki_food_venues['Likes'] = helsinki_food_venues_data['Likes']
helsinki_food_venues['Rating'] = helsinki_food_venues_data['Rating']
helsinki_food_venues['Price tier'] = helsinki_food_venues_data['Price tier']

# drop id column (we needed it only to get more information)
helsinki_food_venues.drop(['id'], axis=1, inplace=True)

# drop rows with NaN values
helsinki_food_venues.dropna(inplace=True)
helsinki_food_venues.reset_index(drop=True, inplace=True)
helsinki_food_venues

,name,categories,lat,lng,postalCode,Tip Count,Likes,Rating,Price tier
0,Café Trocadero,Mediterranean Restaurant,60.165103,24.942498,00120,33,91,9.1,2.0
1,Patisserie Teemu & Markus,Bakery,60.167899,24.938190,00100,30,56,9.0,1.0
2,Penny,American Restaurant,60.164708,24.941308,00120,22,89,9.1,2.0
3,Gaijin,Asian Restaurant,60.165949,24.941759,00120,98,255,8.9,3.0
4,Ravintola Vinkkeli,Scandinavian Restaurant,60.164107,24.946349,00130,13,43,9.2,2.0
5,Passio,Scandinavian Restaurant,60.167088,24.937240,00100,28,92,8.9,2.0
6,Ragu,Restaurant,60.165831,24.944672,00130,31,111,8.7,2.0
7,Ravintola Muru,French Restaurant,60.165268,24.935957,00120,44,133,9.1,3.0
8,Zucchini,Vegetarian / Vegan Restaurant,60.166160,24.949551,00130,15,78,9.0,2.0
9,Boulevard Social,Mediterranean Restaurant,60.166128,24.942034,00120,60,179,8.7,2.0


In [12]:
# We do one hot encoding for categories

helsinki_onehot = pd.get_dummies(helsinki_food_venues[['categories']], prefix="", prefix_sep="")

# add other columns to dataframe
helsinki_onehot['name'] = helsinki_food_venues['name']
helsinki_onehot['Tip Count'] = helsinki_food_venues['Tip Count']
helsinki_onehot['Likes'] = helsinki_food_venues['Likes']
helsinki_onehot['Rating'] = helsinki_food_venues['Rating']
helsinki_onehot['Price tier'] = helsinki_food_venues['Price tier']
helsinki_onehot['lat'] = helsinki_food_venues['lat']
helsinki_onehot['lng'] = helsinki_food_venues['lng']

fixed_columns = list(helsinki_onehot.columns[-7:]) + list(helsinki_onehot.columns[:-7])
helsinki_onehot = helsinki_onehot[fixed_columns]

helsinki_onehot

,name,Tip Count,Likes,Rating,Price tier,lat,lng,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Burger Joint,Burrito Place,Café,Chinese Restaurant,Dumpling Restaurant,Falafel Restaurant,Filipino Restaurant,French Restaurant,Gastropub,Himalayan Restaurant,Japanese Restaurant,Korean Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Pizza Place,Portuguese Restaurant,Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Soup Place,Sushi Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Café Trocadero,33,91,9.1,2.0,60.165103,24.942498,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Patisserie Teemu & Markus,30,56,9.0,1.0,60.167899,24.938190,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Penny,22,89,9.1,2.0,60.164708,24.941308,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Gaijin,98,255,8.9,3.0,60.165949,24.941759,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Ravintola Vinkkeli,13,43,9.2,2.0,60.164107,24.946349,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
5,Passio,28,92,8.9,2.0,60.167088,24.937240,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
6,Ragu,31,111,8.7,2.0,60.165831,24.944672,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
7,Ravintola Muru,44,133,9.1,3.0,60.165268,24.935957,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,Zucchini,15,78,9.0,2.0,60.166160,24.949551,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
9,Boulevard Social,60,179,8.7,2.0,60.166128,24.942034,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
# drop values that we aren't using in k-means and normalize the data
helsinki_grouped_clustering = helsinki_food_venues.drop(['name', 'lat', 'lng', 'postalCode', 'categories'], 1)
X = helsinki_grouped_clustering.values[:,1:]
X = np.nan_to_num(X)
cluster_dataset = StandardScaler().fit_transform(X)
cluster_dataset

array([[-0.25659951,  1.35109901,  0.03771571],
       [-0.59681755,  1.08391538, -1.62177572],
       [-0.27604054,  1.35109901,  0.03771571],
       [ 1.337565  ,  0.81673176,  1.69720714],
       [-0.72318425,  1.61828263,  0.03771571],
       [-0.246879  ,  0.81673176,  0.03771571],
       [-0.06218921,  0.28236451,  0.03771571],
       [ 0.15166213,  1.35109901,  1.69720714],
       [-0.38296621,  1.08391538,  0.03771571],
       [ 0.59880583,  0.28236451,  0.03771571],
       [-0.1205123 ,  1.35109901,  0.03771571],
       [ 2.51374735,  0.81673176,  0.03771571],
       [-0.83010991,  1.61828263,  0.03771571],
       [ 3.88434   ,  0.01518089,  0.03771571],
       [ 1.30840345,  1.61828263,  0.03771571],
       [ 1.44449067,  0.01518089,  0.03771571],
       [ 0.54048274,  0.28236451,  0.03771571],
       [-0.55793549,  0.01518089,  0.03771571],
       [-0.75234579,  1.08391538,  0.03771571],
       [-0.04274818,  1.08391538,  0.03771571],
       [-0.90787404,  0.54954814, -1.621

In [14]:
# set number of clusters
kclusters = 10

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(cluster_dataset)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
helsinki_onehot.insert(0, 'Cluster Labels', kmeans.labels_)

helsinki_onehot

,Cluster Labels,name,Tip Count,Likes,Rating,Price tier,lat,lng,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Burger Joint,Burrito Place,Café,Chinese Restaurant,Dumpling Restaurant,Falafel Restaurant,Filipino Restaurant,French Restaurant,Gastropub,Himalayan Restaurant,Japanese Restaurant,Korean Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Pizza Place,Portuguese Restaurant,Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Soup Place,Sushi Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,2,Café Trocadero,33,91,9.1,2.0,60.165103,24.942498,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7,Patisserie Teemu & Markus,30,56,9.0,1.0,60.167899,24.938190,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,Penny,22,89,9.1,2.0,60.164708,24.941308,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,Gaijin,98,255,8.9,3.0,60.165949,24.941759,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2,Ravintola Vinkkeli,13,43,9.2,2.0,60.164107,24.946349,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
5,2,Passio,28,92,8.9,2.0,60.167088,24.937240,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
6,8,Ragu,31,111,8.7,2.0,60.165831,24.944672,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
7,1,Ravintola Muru,44,133,9.1,3.0,60.165268,24.935957,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,2,Zucchini,15,78,9.0,2.0,60.166160,24.949551,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
9,6,Boulevard Social,60,179,8.7,2.0,60.166128,24.942034,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
# group venues by clusters and take the mean
helsinki_clustered = helsinki_onehot.groupby('Cluster Labels').mean()

# calculate how many venues belongs to each cluster
helsinki_onehot.groupby(['Cluster Labels']).size()
helsinki_clustered['Cluster count'] = helsinki_onehot.groupby(['Cluster Labels']).size()
fixed_columns = [helsinki_clustered.columns[-1]] + list(helsinki_clustered.columns[:-1])
helsinki_clustered = helsinki_clustered[fixed_columns]

helsinki_clustered

,Cluster count,Tip Count,Likes,Rating,Price tier,lat,lng,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Burger Joint,Burrito Place,Café,Chinese Restaurant,Dumpling Restaurant,Falafel Restaurant,Filipino Restaurant,French Restaurant,Gastropub,Himalayan Restaurant,Japanese Restaurant,Korean Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Pizza Place,Portuguese Restaurant,Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Soup Place,Sushi Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
Cluster Labels,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,12,28.583333,71.083333,8.108333,2.000000,60.166806,24.943285,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,0.0000,0.083333,0.000000,0.000000,0.083333,0.0000,0.0000,0.0000,0.083333,0.000000,0.000000,0.083333,0.083333,0.0000,0.000,0.000000,0.000000,0.0000,0.083333,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.083333,0.083333
1,5,55.600000,158.400000,8.960000,3.000000,60.169941,24.931810,0.0000,0.4000,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0000,0.2000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.400000,0.000000,0.000000,0.000000,0.000000,0.000000
2,16,23.437500,78.437500,9.050000,2.000000,60.167972,24.939548,0.0625,0.0625,0.0000,0.000000,0.000000,0.000000,0.0000,0.062500,0.000000,0.000000,0.000000,0.0625,0.0625,0.0000,0.000000,0.000000,0.000000,0.000000,0.062500,0.0000,0.125,0.000000,0.000000,0.0625,0.062500,0.000000,0.000000,0.250000,0.000000,0.062500,0.000000,0.062500,0.000000
3,9,13.111111,37.000000,8.244444,1.000000,60.168325,24.945795,0.0000,0.0000,0.0000,0.000000,0.111111,0.111111,0.0000,0.333333,0.111111,0.111111,0.000000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000,0.000000,0.111111,0.0000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,7,42.142857,125.285714,8.257143,3.142857,60.166746,24.943879,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,0.0000,0.142857,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.000000,0.142857,0.000000,0.000000,0.000000,0.0000,0.000,0.142857,0.000000,0.0000,0.000000,0.000000,0.000000,0.285714,0.000000,0.285714,0.000000,0.000000,0.000000
5,2,138.000000,542.500000,8.350000,2.000000,60.167400,24.944552,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,0.0000,0.500000,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000,0.000000,0.500000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,13,73.307692,218.384615,8.730769,2.000000,60.168695,24.942545,0.0000,0.0000,0.0000,0.076923,0.000000,0.000000,0.0000,0.000000,0.076923,0.000000,0.153846,0.0000,0.0000,0.0000,0.000000,0.000000,0.076923,0.000000,0.076923,0.0000,0.000,0.000000,0.076923,0.0000,0.076923,0.000000,0.076923,0.153846,0.000000,0.076923,0.076923,0.000000,0.000000
7,7,21.714286,64.857143,8.828571,1.000000,60.165734,24.939570,0.0000,0.0000,0.0000,0.000000,0.285714,0.000000,0.0000,0.428571,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.285714,0.000000,0.000000,0.000000,0.000000
8,16,25.437500,85.187500,8.506250,2.000000,60.166797,24.942303,0.0000,0.0000,0.0625,0.000000,0.000000,0.000000,0.0625,0.000000,0.000000,0.000000,0.000000,0.0000,0.0625,0.0625,0.000000,0.062500,0.000000,0.000000,0.000000,0.0625,0.000,0.000000,0.062500,0.0000,0.125000,0.000000,0.000000,0.375000,0.000000,0.000000,0.000000,0.000000,0.062500


In [16]:
# sort by rating
helsinki_clustered.sort_values(by=['Rating'], ascending=False, inplace=True)

# drop 5 clusters that have least rating and cluster label 9, because it includes only one venue
helsinki_clustered.drop(helsinki_clustered.tail(5).index,inplace=True)
helsinki_clustered.drop(helsinki_clustered.head(1).index,inplace=True)
helsinki_clustered

,Cluster count,Tip Count,Likes,Rating,Price tier,lat,lng,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Burger Joint,Burrito Place,Café,Chinese Restaurant,Dumpling Restaurant,Falafel Restaurant,Filipino Restaurant,French Restaurant,Gastropub,Himalayan Restaurant,Japanese Restaurant,Korean Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Pizza Place,Portuguese Restaurant,Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Soup Place,Sushi Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
Cluster Labels,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,16,23.437500,78.437500,9.050000,2.0,60.167972,24.939548,0.0625,0.0625,0.0,0.000000,0.000000,0.0,0.0,0.062500,0.000000,0.0,0.000000,0.0625,0.0625,0.0,0.0,0.0,0.000000,0.0,0.062500,0.0,0.125,0.0,0.000000,0.0625,0.062500,0.0,0.000000,0.250000,0.000000,0.062500,0.000000,0.0625,0.0
1,5,55.600000,158.400000,8.960000,3.0,60.169941,24.931810,0.0000,0.4000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0000,0.2000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000,0.0,0.000000,0.0000,0.000000,0.0,0.000000,0.400000,0.000000,0.000000,0.000000,0.0000,0.0
7,7,21.714286,64.857143,8.828571,1.0,60.165734,24.939570,0.0000,0.0000,0.0,0.000000,0.285714,0.0,0.0,0.428571,0.000000,0.0,0.000000,0.0000,0.0000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000,0.0,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.285714,0.000000,0.000000,0.0000,0.0
6,13,73.307692,218.384615,8.730769,2.0,60.168695,24.942545,0.0000,0.0000,0.0,0.076923,0.000000,0.0,0.0,0.000000,0.076923,0.0,0.153846,0.0000,0.0000,0.0,0.0,0.0,0.076923,0.0,0.076923,0.0,0.000,0.0,0.076923,0.0000,0.076923,0.0,0.076923,0.153846,0.000000,0.076923,0.076923,0.0000,0.0


In [17]:
# Take only top 5 clusters by Rating excluding cluster label 9, because it includes only one venue
helsinki_onehot_4clusters = helsinki_onehot[(helsinki_onehot['Cluster Labels'] != 0) & (helsinki_onehot['Cluster Labels'] != 3) & (helsinki_onehot['Cluster Labels'] != 4) & (helsinki_onehot['Cluster Labels'] != 5) & (helsinki_onehot['Cluster Labels'] != 8)& (helsinki_onehot['Cluster Labels'] != 9)]
helsinki_onehot_4clusters.sort_values(by=['Rating'], ascending=False, inplace=True)
helsinki_onehot_4clusters

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Cluster Labels,name,Tip Count,Likes,Rating,Price tier,lat,lng,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Burger Joint,Burrito Place,Café,Chinese Restaurant,Dumpling Restaurant,Falafel Restaurant,Filipino Restaurant,French Restaurant,Gastropub,Himalayan Restaurant,Japanese Restaurant,Korean Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Pizza Place,Portuguese Restaurant,Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Soup Place,Sushi Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
14,6,Gran Delicato,99,252,9.2,2.0,60.165302,24.932183,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,2,Ravintola Vinkkeli,13,43,9.2,2.0,60.164107,24.946349,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
78,2,Sinisen huvilan kahvila,34,143,9.2,2.0,60.181305,24.937043,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12,2,Cafe Rouge,11,32,9.2,2.0,60.168711,24.933027,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
0,2,Café Trocadero,33,91,9.1,2.0,60.165103,24.942498,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10,2,Levant,25,105,9.1,2.0,60.164131,24.936531,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
39,2,Paulette,26,60,9.1,2.0,60.158475,24.946111,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29,2,Twisted Street Kitchen,23,95,9.1,2.0,60.170641,24.928203,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,1,Ravintola Muru,44,133,9.1,3.0,60.165268,24.935957,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
86,6,Lemon Grass,70,235,9.1,2.0,60.182339,24.949199,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [18]:
# create map for top 4 clusters
map_clusters = folium.Map(location=[latitude_helsinki,longitude_helsinki], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array] 

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(helsinki_onehot_4clusters['lat'], helsinki_onehot_4clusters['lng'], helsinki_onehot_4clusters['name'], helsinki_onehot_4clusters['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [19]:
# we want to take top 10 restaurants of all the venues to compare the results with the clustered venues
helsinki_onehot.sort_values(by=['Rating'], ascending=False, inplace=True)
helsinki_onehot_top10 = helsinki_onehot[0:10]
helsinki_onehot_top10

,Cluster Labels,name,Tip Count,Likes,Rating,Price tier,lat,lng,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Burger Joint,Burrito Place,Café,Chinese Restaurant,Dumpling Restaurant,Falafel Restaurant,Filipino Restaurant,French Restaurant,Gastropub,Himalayan Restaurant,Japanese Restaurant,Korean Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Pizza Place,Portuguese Restaurant,Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Soup Place,Sushi Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
33,9,Johan & Nyström,146,480,9.3,3.0,60.169025,24.960081,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14,6,Gran Delicato,99,252,9.2,2.0,60.165302,24.932183,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,2,Ravintola Vinkkeli,13,43,9.2,2.0,60.164107,24.946349,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
78,2,Sinisen huvilan kahvila,34,143,9.2,2.0,60.181305,24.937043,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12,2,Cafe Rouge,11,32,9.2,2.0,60.168711,24.933027,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
0,2,Café Trocadero,33,91,9.1,2.0,60.165103,24.942498,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10,2,Levant,25,105,9.1,2.0,60.164131,24.936531,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
39,2,Paulette,26,60,9.1,2.0,60.158475,24.946111,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29,2,Twisted Street Kitchen,23,95,9.1,2.0,60.170641,24.928203,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,1,Ravintola Muru,44,133,9.1,3.0,60.165268,24.935957,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [20]:
# create map for top 10 restaurants
map_clusters = folium.Map(location=[latitude_helsinki,longitude_helsinki], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array] 

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(helsinki_onehot_top10['lat'], helsinki_onehot_top10['lng'], helsinki_onehot_top10['name'], helsinki_onehot_top10['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters